In [2]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from wordcloud import WordCloud, STOPWORDS
from mlxtend.frequent_patterns import apriori, association_rules
from tqdm import tqdm
import random
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from wordcloud import WordCloud
from nltk.util import ngrams
import ast
from textblob import TextBlob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Concatenate, Flatten, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences


pd.set_option('display.max_columns', None)

pio.renderers.default = "notebook"


# Configurando o estilo dos gráficos
sns.set(style="whitegrid")

In [3]:
# Carregando os diferentes arquivos CSV
links_small = pd.read_csv('the-movies-dataset/links_small.csv')
movies_metadata = pd.read_csv('the-movies-dataset/movies_metadata.csv')

/var/folders/2r/p7xt963j61jdwt54k047z_kh0000gn/T/ipykernel_25265/3125572303.py:3: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
# Removendo linhas onde a coluna 'title' possui valores nulos, pois o título é uma informação essencial
movies_metadata = movies_metadata.dropna(subset=['title'])

# Criando uma nova coluna 'year', que contém apenas o ano extraído da coluna 'release_date'
movies_metadata['year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if pd.notnull(x) else np.nan
)

# Convertendo a coluna 'year' para valores numéricos, substituindo valores inválidos (não numéricos) por NaN
movies_metadata['year'] = pd.to_numeric(movies_metadata['year'], errors='coerce')
movies_metadata = movies_metadata.dropna(subset=['year'])

# Extraindo a decada do filme
movies_metadata['decade'] = (movies_metadata['year'] // 10) * 10

# Removendo possíveis linhas duplicadas, se houver mais de uma linha com o mesmo 'id' (coluna identificadora do filme)
movies_metadata = movies_metadata.drop_duplicates(subset='id', keep='first')

# Convertendo a coluna 'budget' e 'revenue' para valores numéricos, substituindo valores inválidos (não numéricos) por NaN
movies_metadata['budget'] = pd.to_numeric(movies_metadata['budget'], errors='coerce')
movies_metadata['revenue'] = pd.to_numeric(movies_metadata['revenue'], errors='coerce')

# No dataset links_small, filtramos para manter apenas as linhas onde a coluna 'tmdbId' não tem valores nulos e convertemos para inteiros
links_small_id = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

# Exibindo a estrutura do dataset movies_metadata e links_small após a limpeza
print("\nEstrutura de Movies Metadata após a limpeza:")
print(movies_metadata.info())

print("\nEstrutura de Links Small após a limpeza:")
print(links_small.info())


Estrutura de Movies Metadata após a limpeza:
<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45346 non-null  object 
 1   belongs_to_collection  4485 non-null   object 
 2   budget                 45346 non-null  int64  
 3   genres                 45346 non-null  object 
 4   homepage               7761 non-null   object 
 5   id                     45346 non-null  object 
 6   imdb_id                45332 non-null  object 
 7   original_language      45335 non-null  object 
 8   original_title         45346 non-null  object 
 9   overview               44405 non-null  object 
 10  popularity             45346 non-null  object 
 11  poster_path            45007 non-null  object 
 12  production_companies   45346 non-null  object 
 13  production_countries   45346 non-null  object 
 14  release_date 

In [5]:
#Removendo linhas com valores nulos e criando uma nova coluna 'ROI' que representa a relação entre a receita e o orçamento
movies_metadata['revenue'] = movies_metadata['revenue'].replace(0, np.nan)
movies_metadata['budget'] = movies_metadata['budget'].replace(0, np.nan)
movies_metadata['ROI'] = movies_metadata['revenue'] / movies_metadata['budget'] - 1
movies_metadata['profit'] = movies_metadata['revenue'] - movies_metadata['budget']

In [6]:
#Filtrando os filmes que tem relação com link_small
movies_metadata['id'] = movies_metadata['id'].astype('int')
movies_metadata = movies_metadata[movies_metadata['id'].isin(links_small_id)]

In [7]:
movies_metadata = pd.merge(movies_metadata, links_small, left_on='id', right_on='tmdbId', how='inner')

In [8]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9082 non-null   object 
 1   belongs_to_collection  1672 non-null   object 
 2   budget                 4639 non-null   float64
 3   genres                 9082 non-null   object 
 4   homepage               1971 non-null   object 
 5   id                     9082 non-null   int64  
 6   imdb_id                9082 non-null   object 
 7   original_language      9082 non-null   object 
 8   original_title         9082 non-null   object 
 9   overview               9070 non-null   object 
 10  popularity             9082 non-null   object 
 11  poster_path            9079 non-null   object 
 12  production_companies   9082 non-null   object 
 13  production_countries   9082 non-null   object 
 14  release_date           9082 non-null   object 
 15  reve

In [9]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,decade,ROI,profit,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995.0,1990.0,11.451801,343554033.0,1,114709,862.0
1,False,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995.0,1990.0,3.043035,197797249.0,2,113497,8844.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,NaN,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995.0,1990.0,NaN,NaN,3,113228,15602.0
3,False,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995.0,1990.0,4.090760,65452156.0,4,114885,31357.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995.0,1990.0,NaN,NaN,5,113041,11862.0


In [10]:
ratings = pd.read_csv('./the-movies-dataset/ratings_small.csv')

In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [12]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [13]:
# Counting the number of occurrences of each user
user_counts = ratings['userId'].value_counts()

# Selecting users who appear the most until the dataset has at most 1000 rows
selected_users = []
total_rows = 0

for user, count in user_counts.items():
    if count > 100:
        continue
    if total_rows + count > 10000:
        break
    selected_users.append(user)
    total_rows += count

# Filtering the dataset to only include the selected users
ratings = ratings[ratings['userId'].isin(selected_users)]

In [14]:
# Preencher 'budget' e 'revenue' com a mediana
movies_metadata['budget'] = movies_metadata['budget'].fillna(movies_metadata['budget'].median())
movies_metadata['revenue'] = movies_metadata['revenue'].fillna(movies_metadata['revenue'].median())

# Preencher 'runtime' com a mediana
movies_metadata['runtime'] = movies_metadata['runtime'].fillna(movies_metadata['runtime'].median())

# Preencher 'vote_average' e 'vote_count' com a mediana
movies_metadata['vote_average'] = movies_metadata['vote_average'].fillna(movies_metadata['vote_average'].median())
movies_metadata['vote_count'] = movies_metadata['vote_count'].fillna(movies_metadata['vote_count'].median())

# Preencher 'year' e 'decade' com a mediana
movies_metadata['year'] = movies_metadata['year'].fillna(movies_metadata['year'].median())
movies_metadata['decade'] = movies_metadata['decade'].fillna(movies_metadata['decade'].median())

# 'ROI' e 'profit' possuem muitos nulos. Podemos preenchê-los com zero ou deixar como estão, dependendo da importância.
movies_metadata['ROI'] = movies_metadata['ROI'].fillna(0)
movies_metadata['profit'] = movies_metadata['profit'].fillna(0)

In [15]:
import ast

# Função para extrair nomes de uma coluna que contém listas de dicionários
def extract_names(list_str, key='name'):
    try:
        items = ast.literal_eval(list_str)
        return [item[key] for item in items]
    except (ValueError, SyntaxError):
        return []

# Extrair gêneros
movies_metadata['genres_list'] = movies_metadata['genres'].apply(lambda x: extract_names(x, 'name'))

# Extrair nomes das companhias de produção
movies_metadata['production_companies_list'] = movies_metadata['production_companies'].apply(lambda x: extract_names(x, 'name'))

# Extrair nomes dos países de produção
movies_metadata['production_countries_list'] = movies_metadata['production_countries'].apply(lambda x: extract_names(x, 'name'))

# Extrair nomes das línguas faladas
movies_metadata['spoken_languages_list'] = movies_metadata['spoken_languages'].apply(lambda x: extract_names(x, 'name'))

In [16]:
# Identificar os gêneros mais frequentes
all_genres = movies_metadata['genres_list'].explode()
top_genres = all_genres.value_counts().index[:20]  # Selecionar top 20 gêneros

# Criar colunas binárias para os top gêneros
for genre in top_genres:
    movies_metadata[f'genre_{genre}'] = movies_metadata['genres_list'].apply(lambda x: 1 if genre in x else 0)

# Contar o número de companhias de produção
movies_metadata['num_production_companies'] = movies_metadata['production_companies_list'].apply(len)

In [17]:
# Contar o número de países de produção
movies_metadata['num_production_countries'] = movies_metadata['production_countries_list'].apply(len)

In [18]:
# Contar o número de línguas faladas
movies_metadata['num_spoken_languages'] = movies_metadata['spoken_languages_list'].apply(len)

In [19]:
# Converter 'release_date' para datetime, se ainda não estiver
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')

# Extrair ano de lançamento a partir de 'release_date'
movies_metadata['release_year'] = movies_metadata['release_date'].dt.year
movies_metadata['release_year'] = movies_metadata['release_year'].fillna(movies_metadata['release_year'].median())

In [20]:
from sklearn.preprocessing import LabelEncoder

# Codificar 'original_language' usando Label Encoding
le_language = LabelEncoder()
movies_metadata['original_language_encoded'] = le_language.fit_transform(movies_metadata['original_language'])



In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Preencher 'overview' com string vazia se ainda não estiver feito
movies_metadata['overview'] = movies_metadata['overview'].fillna('')

# Limitar o número de features para evitar alta dimensionalidade
tfidf = TfidfVectorizer(max_features=40)
overview_tfidf = tfidf.fit_transform(movies_metadata['overview']).toarray()

# Criar um DataFrame com os features TF-IDF
tfidf_df = pd.DataFrame(overview_tfidf, columns=[f'overview_tfidf_{i}' for i in range(overview_tfidf.shape[1])])

# Concatenar com o DataFrame principal
movies_metadata = pd.concat([movies_metadata, tfidf_df], axis=1)

In [22]:
movies_metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,decade,ROI,profit,movieId,imdbId,tmdbId,genres_list,production_companies_list,production_countries_list,spoken_languages_list,genre_Drama,genre_Comedy,genre_Thriller,genre_Romance,genre_Action,genre_Crime,genre_Adventure,genre_Horror,genre_Science Fiction,genre_Family,genre_Fantasy,genre_Mystery,genre_Documentary,genre_Music,genre_Animation,genre_History,genre_War,genre_Western,genre_Foreign,genre_TV Movie,num_production_companies,num_production_countries,num_spoken_languages,release_year,original_language_encoded,overview_tfidf_0,overview_tfidf_1,overview_tfidf_2,overview_tfidf_3,overview_tfidf_4,overview_tfidf_5,overview_tfidf_6,overview_tfidf_7,overview_tfidf_8,overview_tfidf_9,overview_tfidf_10,overview_tfidf_11,overview_tfidf_12,overview_tfidf_13,overview_tfidf_14,overview_tfidf_15,overview_tfidf_16,overview_tfidf_17,overview_tfidf_18,overview_tfidf_19,overview_tfidf_20,overview_tfidf_21,overview_tfidf_22,overview_tfidf_23,overview_tfidf_24,overview_tfidf_25,overview_tfidf_26,overview_tfidf_27,overview_tfidf_28,overview_tfidf_29,overview_tfidf_30,overview_tfidf_31,overview_tfidf_32,overview_tfidf_33,overview_tfidf_34,overview_tfidf_35,overview_tfidf_36,overview_tfidf_37,overview_tfidf_38,overview_tfidf_39
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995.0,1990.0,11.451801,343554033.0,1,114709,862.0,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English],0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,1,1995,10,0.000000,0.000000,0.142352,0.000000,0.000000,0.000000,0.0,0.278393,0.263246,0.000000,0.274742,0.000000,0.000000,0.0,0.0,0.398544,0.334430,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.148197,0.000000,0.0,0.000000,0.000000,0.000000,0.255140,0.556123,0.000000,0.0,0.141996,0.0,0.00000,0.265097,0.000000,0.000000,0.0
1,False,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995.0,1990.0,3.043035,197797249.0,2,113497,8844.0,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]",0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,3,1,2,1995,10,0.000000,0.402778,0.237837,0.000000,0.217638,0.000000,0.0,0.000000,0.000000,0.399373,0.229515,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.273807,0.163833,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.216192,0.319709,0.232287,0.258274,0.0,0.237242,0.0,0.00000,0.221457,0.211911,0.000000,0.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",18500000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,33468409.5,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Sti

In [23]:
# Seleção de colunas
selected_columns = [
    'movieId',
    'budget',
    'revenue',
    'runtime',
    'vote_average',
    'vote_count',
    'year',
    'decade',
    'ROI',
    'profit',
    'original_language_encoded',
    'release_year',
    'num_production_companies',
    'num_production_countries',
    'num_spoken_languages'
]  + [f'genre_{genre}' for genre in top_genres]

selected_columns += list(tfidf_df.columns)

# Verificar as colunas selecionadas
print("Colunas selecionadas para o modelo:")
print(selected_columns)

Colunas selecionadas para o modelo:
['movieId', 'budget', 'revenue', 'runtime', 'vote_average', 'vote_count', 'year', 'decade', 'ROI', 'profit', 'original_language_encoded', 'release_year', 'num_production_companies', 'num_production_countries', 'num_spoken_languages', 'genre_Drama', 'genre_Comedy', 'genre_Thriller', 'genre_Romance', 'genre_Action', 'genre_Crime', 'genre_Adventure', 'genre_Horror', 'genre_Science Fiction', 'genre_Family', 'genre_Fantasy', 'genre_Mystery', 'genre_Documentary', 'genre_Music', 'genre_Animation', 'genre_History', 'genre_War', 'genre_Western', 'genre_Foreign', 'genre_TV Movie', 'overview_tfidf_0', 'overview_tfidf_1', 'overview_tfidf_2', 'overview_tfidf_3', 'overview_tfidf_4', 'overview_tfidf_5', 'overview_tfidf_6', 'overview_tfidf_7', 'overview_tfidf_8', 'overview_tfidf_9', 'overview_tfidf_10', 'overview_tfidf_11', 'overview_tfidf_12', 'overview_tfidf_13', 'overview_tfidf_14', 'overview_tfidf_15', 'overview_tfidf_16', 'overview_tfidf_17', 'overview_tfidf_18

In [24]:
movies_metadata = movies_metadata[selected_columns]

In [25]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 75 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movieId                    9082 non-null   int64  
 1   budget                     9082 non-null   float64
 2   revenue                    9082 non-null   float64
 3   runtime                    9082 non-null   float64
 4   vote_average               9082 non-null   float64
 5   vote_count                 9082 non-null   float64
 6   year                       9082 non-null   float64
 7   decade                     9082 non-null   float64
 8   ROI                        9082 non-null   float64
 9   profit                     9082 non-null   float64
 10  original_language_encoded  9082 non-null   int64  
 11  release_year               9082 non-null   int32  
 12  num_production_companies   9082 non-null   int64  
 13  num_production_countries   9082 non-null   int64

In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9999 entries, 20 to 99800
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     9999 non-null   int64  
 1   movieId    9999 non-null   int64  
 2   rating     9999 non-null   float64
 3   timestamp  9999 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 390.6 KB


# Aqui começa o modelo

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Concatenate, Flatten, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.backend as K
from tensorflow.keras import backend as K

# Supondo que 'ratings' e 'movies_metadata' já estejam carregados

# 1. Mapeamento de IDs para Índices
user_ids = ratings['userId'].unique()
user2idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
ratings['user_idx'] = ratings['userId'].map(user2idx)

movie_ids = movies_metadata['movieId'].unique()
movie2idx = {movie_id: idx + 1 for idx, movie_id in enumerate(movie_ids)}  # Inicia em 1 para padding
ratings['movie_idx'] = ratings['movieId'].map(movie2idx)

# Substitua filmes desconhecidos por 0 (padding)
ratings['movie_idx'] = ratings['movie_idx'].fillna(0).astype(int)

print(f'Número de usuários únicos: {len(user2idx)}')
print(f'Número de filmes únicos: {len(movie2idx)}')

# 2. Transformação das Notas para a Escala -1 a 1
original_min = ratings['rating'].min()  # Supondo 0.5
original_max = ratings['rating'].max()  # Supondo 5.0

def scale_rating(rating, original_min=0.5, original_max=5.0):
    return 2 * ((rating - original_min) / (original_max - original_min)) - 1

ratings['rating_scaled'] = ratings['rating'].apply(scale_rating)
print(ratings[['rating', 'rating_scaled']].head())

# 3. Agregação do Histórico de Avaliações dos Usuários
sequence_length = 20
ratings_sorted = ratings.sort_values(['userId', 'timestamp'])
user_histories = ratings_sorted.groupby('userId')['movie_idx'].apply(list).to_dict()

def create_history(user_id, user_histories, sequence_length=20):
    history = user_histories.get(user_id, [])
    if len(history) >= sequence_length:
        return history[-sequence_length:]
    else:
        # Pad com zeros no início
        return [0] * (sequence_length - len(history)) + history

ratings['history'] = ratings['userId'].apply(lambda x: create_history(x, user_histories, sequence_length))
history_sequences = np.array(ratings['history'].tolist())
print(f'Exemplo de sequência de histórico: {history_sequences[0]}')

# 4. Integração das Características dos Filmes
non_feature_columns = ['movieId', 'title', 'overview','overview_tfidf_0', 'overview_tfidf_1', 'overview_tfidf_2', 'overview_tfidf_3', 'overview_tfidf_4', 'overview_tfidf_5', 'overview_tfidf_6', 'overview_tfidf_7', 'overview_tfidf_8', 'overview_tfidf_9', 'overview_tfidf_10', 'overview_tfidf_11', 'overview_tfidf_12', 'overview_tfidf_13', 'overview_tfidf_14', 'overview_tfidf_15', 'overview_tfidf_16', 'overview_tfidf_17', 'overview_tfidf_18', 'overview_tfidf_19', 'overview_tfidf_20', 'overview_tfidf_21', 'overview_tfidf_22', 'overview_tfidf_23', 'overview_tfidf_24', 'overview_tfidf_25', 'overview_tfidf_26', 'overview_tfidf_27', 'overview_tfidf_28', 'overview_tfidf_29', 'overview_tfidf_30', 'overview_tfidf_31', 'overview_tfidf_32', 'overview_tfidf_33', 'overview_tfidf_34', 'overview_tfidf_35', 'overview_tfidf_36', 'overview_tfidf_37', 'overview_tfidf_38', 'overview_tfidf_39']  # Removi 'other_non_feature_columns'
feature_columns = [col for col in movies_metadata.columns if col not in non_feature_columns]

print(f'Número de features dos filmes: {len(feature_columns)}')
print('Colunas de features dos filmes:', feature_columns)

# Verificar se há duplicatas
duplicate_features = [feature for feature in feature_columns if feature_columns.count(feature) > 1]
print(f'Features duplicadas: {duplicate_features}')

# Normalização das Features Numéricas
numeric_features = ['revenue', 'runtime', 'vote_average', 'vote_count']  # Atualize conforme necessário
scaler = StandardScaler()
movies_metadata[numeric_features] = scaler.fit_transform(movies_metadata[numeric_features])

# Verificação da normalização
print(movies_metadata[numeric_features].head())

# Preparação das Features dos Filmes
movies_features = movies_metadata.set_index('movieId')[feature_columns]

def get_movie_features(movie_id):
    if movie_id in movies_features.index:
        return movies_features.loc[movie_id].values
    else:
        return np.zeros(len(feature_columns))

ratings['movie_features'] = ratings['movieId'].apply(get_movie_features)
movie_features_array = np.array(ratings['movie_features'].tolist())
print(f'Exemplo de features de filme: {movie_features_array[0]}')
print(f'Dimensionalidade das features dos filmes: {movie_features_array.shape[1]}')  # Deve ser igual a len(feature_columns)

# Verifique se feature_columns está correto e não tem duplicatas
print(f'Número de feature_columns: {len(feature_columns)}')
print(f'movie_features_array shape: {movie_features_array.shape}')

# 5. Definir tamanhos dos embeddings
user_embedding_size = 50
movie_embedding_size = 50

# Inputs
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')
history_input = Input(shape=(sequence_length,), name='history_input')
features_input = Input(shape=(len(feature_columns),), name='features_input')  # 74

# Camadas de Embedding
user_embedding_layer = Embedding(input_dim=len(user2idx), output_dim=user_embedding_size, name='user_embedding')(user_input)
movie_embedding_layer = Embedding(input_dim=len(movie2idx) + 1, output_dim=movie_embedding_size, name='movie_embedding')(movie_input)  # +1 para padding
history_embedding_layer = Embedding(input_dim=len(movie2idx) + 1, output_dim=movie_embedding_size, name='history_embedding')(history_input)

# Flatten das embeddings
user_vec = Flatten()(user_embedding_layer)  # (None, 50)
movie_vec = Flatten()(movie_embedding_layer)  # (None, 50)
history_vec = LSTM(128)(history_embedding_layer)  # (None, 128)

print(f'Formato da camada de usuário: {user_vec.shape}')
print(f'Formato da camada de filme: {movie_vec.shape}')
print(f'Formato da camada de histórico: {history_vec.shape}')

# Concatenar as embeddings dos usuários, histórico, filmes e features
combined = Concatenate()([user_vec, movie_vec, history_vec, features_input])  # Esperado: (None, 302)

print(f'Formato da camada combinada: {combined.shape}')

# Camadas Densas
dense1 = Dense(128, activation='relu')(combined)
dense2 = Dense(64, activation='relu')(dense1)
output = Dense(1, activation='tanh')(dense2)

# Modelo Final
model = Model(inputs=[user_input, movie_input, history_input, features_input], outputs=output)

# Compilação do Modelo
model.compile(optimizer=Adam(learning_rate = 0.0001), loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error'])

# Resumo do Modelo
model.summary()

# Divisão dos Dados
X = np.hstack([ratings['user_idx'].values.reshape(-1, 1), 
               ratings['movie_idx'].values.reshape(-1, 1), 
               history_sequences, 
               movie_features_array])
y = ratings['rating_scaled'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do Modelo	
history = model.fit([X_train[:, 0].reshape(-1, 1), 
                     X_train[:, 1].reshape(-1, 1), 
                     X_train[:, 2:22], 
                     X_train[:, 22:96]], 
                    y_train, 
                    batch_size=64, 
                    epochs=100, 
                    validation_data=([X_test[:, 0].reshape(-1, 1), 
                                      X_test[:, 1].reshape(-1, 1), 
                                      X_test[:, 2:22], 
                                      X_test[:, 22:96]], 
                                     y_test))

# Avaliação do Modelo
results = model.evaluate([X_test[:, 0].reshape(-1, 1), 
                          X_test[:, 1].reshape(-1, 1), 
                          X_test[:, 2:22], 
                          X_test[:, 22:96]], 
                         y_test, 
                         batch_size=64)

print('Test Loss:', results[0])
print('Test MAE:', results[1])
print('Test MSE:', results[2])

Número de usuários únicos: 132
Número de filmes únicos: 9082
    rating  rating_scaled
20     4.0       0.555556
21     5.0       1.000000
22     5.0       1.000000
23     4.0       0.555556
24     4.0       0.555556
Exemplo de sequência de histórico: [196 455 496 445 221 286 433 335 544 336   0 124  50 281 337 494 568 347
 482 358]
Número de features dos filmes: 34
Colunas de features dos filmes: ['budget', 'revenue', 'runtime', 'vote_average', 'vote_count', 'year', 'decade', 'ROI', 'profit', 'original_language_encoded', 'release_year', 'num_production_companies', 'num_production_countries', 'num_spoken_languages', 'genre_Drama', 'genre_Comedy', 'genre_Thriller', 'genre_Romance', 'genre_Action', 'genre_Crime', 'genre_Adventure', 'genre_Horror', 'genre_Science Fiction', 'genre_Family', 'genre_Fantasy', 'genre_Mystery', 'genre_Documentary', 'genre_Music', 'genre_Animation', 'genre_History', 'genre_War', 'genre_Western', 'genre_Foreign', 'genre_TV Movie']
Features duplicadas: []
    reve

/var/folders/2r/p7xt963j61jdwt54k047z_kh0000gn/T/ipykernel_25265/1310303707.py:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Exemplo de features de filme: [1.07484658e+00 2.29424885e+00 8.01955549e-01 2.28963259e-01
 7.57903705e-01 1.99500000e+03 1.99000000e+03 5.07231093e+00
 2.94194034e+08 1.00000000e+01 1.99500000e+03 2.00000000e+00
 2.00000000e+00 3.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Dimensionalidade das features dos filmes: 34
Número de feature_columns: 34
movie_features_array shape: (9999, 34)
Formato da camada de usuário: (None, 50)
Formato da camada de filme: (None, 50)
Formato da camada de histórico: (None, 128)
Formato da camada combinada: (None, 262)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ history_input       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 50)     │      6,600 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_embedding     │ (None, 1, 50)     │    454,150 │ movie_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ history_embedding   │ (None, 20, 50)    │    454,150 │ history_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 50)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 50)        │          0 │ movie_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 128)       │     91,648 │ history_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ features_input      │ (None, 34)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 262)       │          0 │ flatten_4[0][0],  │
│ (Concatenate)       │                   │            │ flatten_5[0][0],  │
│                     │                   │            │ lstm_2[0][0],     │
│                     │                   │            │ features_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     33,664 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │         65 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

...

 Trainable params: 1,048,533 (4.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


/Users/pedro-pc/Developer/INF1038---APRENDIZADO-AUTOMATICO/venv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning:

The structure of `inputs` doesn't match the expected structure: ['user_input', 'movie_input', 'history_input', 'features_input']. Received: the structure of inputs=('*', '*', '*', '*')



125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.5205 - mean_absolute_error: 0.5559 - mean_squared_error: 0.5205 - val_loss: 0.4820 - val_mean_absolute_error: 0.5389 - val_mean_squared_error: 0.4820
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.4963 - mean_absolute_error: 0.5456 - mean_squared_error: 0.4963 - val_loss: 0.4845 - val_mean_absolute_error: 0.5399 - val_mean_squared_error: 0.4845
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.8813 - mean_absolute_error: 1.2341 - mean_squared_error: 1.8813 - val_loss: 1.9688 - val_mean_absolute_error: 1.2814 - val_mean_squared_error: 1.9688
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.8630 - mean_absolute_error: 0.7258 - mean_squared_error: 0.8630 - val_loss: 0.4827 - val_mean_absolute_error: 0.5384 - val_mean_squared_error: 0.4827
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.4985 - mean_absolute_error: 0.5444 - mean_squared_error: 0.4985 - val_loss: 0.4825 - val_mea

In [28]:
# Encode target labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_classes)

# One-hot encode the target classes for neural network compatibility
y_categorical = to_categorical(y_encoded)

# Split data (already defined in your code)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

# Define a neural network model
def create_nn_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))  # Adding dropout for regularization
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_categorical.shape[1], activation='softmax'))  # Softmax for multi-class classification
    return model

# Initialize and compile the model
input_dim = X_train.shape[1]
nn_model = create_nn_model(input_dim)
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test data
y_pred_proba = nn_model.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_proba.argmax(axis=1))
y_true = label_encoder.inverse_transform(y_test.argmax(axis=1))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

# Print results
print(f"\nResultados para Neural Network:")
print(f"Acurácia: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")

# Confusion Matrix
plt.figure(figsize=(8, 6))
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='coolwarm', cbar=False)
plt.title('Matriz de Confusão')
plt.ylabel('Classe Verdadeira')
plt.xlabel('Classe Prevista')
plt.show()

NameError: name 'y_classes' is not defined